<a href="https://colab.research.google.com/github/fwasim/fake_news_classification_BERT/blob/master/BERT_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

import xml.etree.ElementTree as ET
import pandas as pd 
#from google.colab import files
import io
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Uploading file
#uploaded = files.upload()

Saving test.text.tsv to test.text.tsv


In [ ]:
'''tree = ET.parse(io.BytesIO(uploaded["articles-training-byarticle-20181122.xml"]))
root = tree.getroot()

print('Root\tAttributes\n')
for article in root.findall('article'):
  print(article.tag + '\t' + str(article.attrib) + '\n')
  sentences = []
  for text in article.itertext():
    proc_text = text.replace('&amp;', 'and').strip(' \n')
    if len(proc_text) > 0:
      sentences.append(proc_text)
  print(sentences)
  print('\n')'''

In [ ]:
df = pd.read_csv("./test.text.tsv",sep="\t")
#df = pd.read_csv(io.BytesIO(uploaded["test.text.tsv"]), sep="\t")
df = df.drop(['Index', '1', '2'], axis=1) # Dropping columns not needed
pruned_text = []

def remove_link(text):  # Function to remove links from the articles
  output = text
  first_index = output.find('http')
  while first_index > -1:
    last_index = output.find(' ', first_index)
    output = output[0:first_index] + output[last_index:len(output) - 1]
    first_index = output.find('http')
  return output

def remove_char(text, char):  # Function to remove any unwanted char sequence from the articles
  output = text
  first_index = output.find(char)
  while first_index > -1:
    output = output.replace(char, '')
    first_index = output.find(char)
  return output

for text in df['Body']:
  remove_token = str(text).replace('<splt>', '')  # Removing specialized tokens added by the preprocessing script that are not needed for our purpose

  remove_html_tags = remove_token
  if remove_html_tags.count('<') > 0:
    if (remove_html_tags.find('< iframe') != -1 and remove_html_tags.find('</iframe >') != -1): # Removing iframe tags
      first_index = remove_html_tags.find('< iframe')
      last_index = remove_html_tags.find('</iframe >')
      remove_html_tags = str(remove_html_tags)[0:first_index] + str(remove_html_tags)[last_index + 10:len(remove_html_tags) - 1]
    if (remove_html_tags.find('< script') != -1 and remove_html_tags.find('</script >') != -1): # Removing script tags
      first_index = remove_html_tags.find('< script')
      last_index = remove_html_tags.find('</script >')
      remove_html_tags = str(remove_html_tags)[0:first_index] + str(remove_html_tags)[last_index + 10:len(remove_html_tags) - 1]
  if remove_html_tags.count('>') > 0: # Removing anomalous '>' characters
    remove_html_tags = remove_char(remove_html_tags, '>')
  if (remove_html_tags.count('http') > -1): # Removing links
    remove_html_tags = remove_link(remove_html_tags)
  
  pruned_text.append(remove_html_tags)

df['Body'] = pruned_text
    


In [ ]:
df['Target'].value_counts()

False    407
True     238
Name: Target, dtype: int64

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')  # Using the 'distilbert-base-uncased' pretrained weights

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
title = np.array(df['Title'])
body = np.array(df['Body'])
concat_array  = []

for text in title:
  index = np.where(title == text)[0][0]
  concat_array.append(text + '. ' + body[index])

concat_array = pd.Series(concat_array)
tokenized = concat_array.apply((lambda x: tokenizer.encode(x[0:512], add_special_tokens=True))) # Tokenizing the first 512 characters of the title concatenated to the body of the article
#tokenized = df['Body'].apply((lambda x: tokenizer.encode(x[0:512], add_special_tokens=True))) # Tokenizing the first 512 characters the body of the article
#print(tokenized.values)

[list([101, 13970, 27085, 2818, 1024, 24104, 2075, 1996, 2769, 2373, 1012, 2013, 17312, 2099, 1012, 4012, 1024, 2769, 1063, 3054, 1011, 17365, 2581, 2683, 2509, 1065, 2769, 1006, 3746, 2011, 22649, 1006, 1047, 1007, 2286, 1007, 6656, 4751, 1040, 12458, 2050, 2053, 17357, 2064, 2644, 9543, 3490, 5809, 1010, 2021, 2023, 1015, 6881, 7577, 2064, 1996, 3681, 2024, 5494, 1999, 2006, 3519, 1012, 17082, 3681, 1997, 2300, 2013, 17035, 7702, 1998, 20868, 2863, 1010, 2029, 1010, 2043, 1996, 4053, 2003, 23596, 1010, 2071, 4125, 2000, 1037, 2431, 23458, 6363, 1012, 1996, 3681, 1997, 2162, 1024, 1996, 4800, 1011, 23458, 7922, 7552, 3465, 1997, 7041, 1010, 5712, 1998, 2060, 19388, 1012, 1996, 24827, 3681, 1997, 1996, 1057, 1012, 1055, 1012, 1045, 102])
 list([101, 8398, 2074, 8271, 2039, 1004, 13925, 2135, 4457, 5984, 13641, 2015, 2006, 10474, 2066, 1037, 10311, 2214, 2158, 1012, 6221, 8398, 2743, 2006, 2116, 23678, 9365, 9793, 2015, 1998, 4321, 4895, 22852, 6553, 3049, 10659, 1012, 2028, 1997, 2216,

In [ ]:
max_len = 0
for i in tokenized.values:  # Finding the max length of the token arrays
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values]) # Padding all the token arrays up to the size of the longest one

In [ ]:
np.array(padded).shape

(645, 445)

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)  # Creating an attention mask to tell the model to ignore the padding
attention_mask.shape

(645, 445)

In [ ]:
input_ids = torch.tensor(padded)  # Converting the tokenized, padded vectors to a tensor
attention_mask = torch.tensor(attention_mask) # Converting the attention mask to a tensor

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)  # Fine tuning the model

In [ ]:
features = last_hidden_states[0][:,0,:].numpy() # Taking the [CLS] token of the last hidden layer of all sentences 
labels = df['Target']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels) # Creating a train test split

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)  # Passing the results from last step to logistic regression layer

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(test_features, test_labels)  # Score

0.7901234567901234